# HKU QIDS 2023 Quantitative Investment Competition

## Init Config

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from qids_package.qids import *
import warnings
from submit import submit

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")
seed = 257248

In [2]:
def train_valid_split(data_set, valid_ratio=0.2, seed=257248):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set, valid_set = train_test_split(data_set, test_size=valid_set_size, random_state=seed)
    return train_set, valid_set

In [10]:
def std(train, valid, test=None):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    valid = scaler.transform(valid)
    if test is not None:
        test = scaler.transform(test)

## Load Data

In [11]:
train_fun = pd.read_csv("../data/first_round_train_fundamental_data.csv")
train_mar = pd.read_csv("../data/first_round_train_market_data.csv")
train_ret = pd.read_csv("../data/first_round_train_return_data.csv")

In [12]:
train_fun

,date_time,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf
0,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700
1,s1d1,2.5150,3706.0,28.9934,27.2726,5.0552,3.0484,23.8260
2,s2d1,1.2858,5136.0,42.9352,41.9279,4.8083,4.1392,-58.2185
3,s3d1,2.2007,3280.0,15.2245,13.8032,2.1904,0.6691,61.0491
4,s4d1,0.8627,5291.0,-369.9850,-433.1736,3.0714,2.9370,-25.2279
...,...,...,...,...,...,...,...,...
53995,s49d1000,0.3201,4603.0,40.4933,40.4933,2.7886,1.0333,72.0836
53996,s50d1000,0.4137,2514.0,19.7194,19.7194,2.5400,1.1739,-38.5837
53997,s51d1000,1.1524,5514.0,13.8265,13.8265,1.3388,0.6641,18.4961
53998,s52d1000,0.6358,7556.0,28.5368,28.5368,5.6946,3.6566,-573.3135


In [13]:
train_mar

,date_time,open,close,high,low,volume,money
0,s0d1p1,25.0164,24.9557,25.1378,24.9436,642243.0,1.610060e+07
1,s1d1p1,16.3256,16.3499,16.4106,16.3256,15552.0,2.540301e+05
2,s2d1p1,9.2006,9.1763,9.2006,9.1521,396647.0,3.638304e+06
3,s3d1p1,9.1521,9.2006,9.2006,9.1521,317044.0,2.907092e+06
4,s4d1p1,5.1829,5.1222,5.1829,5.1101,294784.0,1.514496e+06
...,...,...,...,...,...,...,...
2699995,s49d1000p50,13.7524,13.7402,13.7524,13.7402,147800.0,2.030253e+06
2699996,s50d1000p50,11.4340,11.4219,11.4340,11.4219,13000.0,1.484235e+05
2699997,s51d1000p50,3.5200,3.5200,3.5200,3.5200,284775.0,1.002944e+06
2699998,s52d1000p50,28.4151,28.4151,28.4151,28.4151,22654.0,6.437413e+05


In [4]:
test_fun = pd.read_csv("./qids_package/first_round_test_fundamental_data.csv")
test_mar = pd.read_csv("./qids_package/first_round_test_market_data.csv")

In [5]:
train_fun, valid_fun = train_valid_split(train_fun)
train_mar, valid_mar = train_valid_split(train_mar)
train_ret, valid_ret = train_valid_split(train_ret)

## Preprocessing

In [ ]:
std(train, valid, test)

## Submission